In [ ]:
# Install required packages
!pip install pyenchant && sudo apt-get install python-enchant
!pip install afinn
!pip install langdetect

# Download spacy model
import spacy.cli
spacy.cli.download("en_core_web_lg")

# Import libraries used
import enchant
import unicodedata
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spacy
import en_core_web_lg
import re
import nltk
from langdetect import detect
from langdetect import DetectorFactory
from langdetect import detect
import nltk
nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('vader_lexicon')
from afinn import Afinn
 
nlp = en_core_web_lg.load()
af = Afinn()
senti = SentimentIntensityAnalyzer()
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/Thesis/")
!ls

     |████████████████████████████████| 61kB 3.7MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko python-gobject python-gtk2
  python-wxgtk3.0
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
  python-enchant
0 upgraded, 11 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,354 kB of archives.
After this operation, 5,584 kB of ad

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
Mounted at /content/drive
 Archive			    rfe_grid.csv
 cleaned_reviews_data_lang.csv	    Ridge_grid2.csv
 cleaned_reviews_data.pickle	    Ridge_grid.csv
 cleaned_reviews_en.csv		    Sample_2.ipynb
 cleaned_reviews_en_lemma.csv	    Sample.ipynb
 clustered_data.pickle		    score_pd
'clustered location.ipynb'	    SentimentAnalysis_2.ipynb
 complete_data_merged.csv	   'SENTIMENT ANALYSIS FINAL.ipynb'
'Cópia de cleaned_reviews_en.csv'   single_character_pd
'Cópia de Sample.ipynb'		    spell_pd
'JOIN SENTIMENT.ipynb'		    SVR_grid2.csv
 Linear_grid2.csv		    SVR_grid.csv
 Linear_grid.csv		   'Text Pre-Processing.ipynb'
 listings.csv			    Untitled0.ipynb
 listings.csv.gz		    Untitled1.ipynb
 model.png			    words_df.pickle
 negative-words.txt		    words_pd
 new_listings.csv		    x_data.pickle
 new_listings.pickle		    XGB_grid2.csv
 positive-words.txt		    XGB_grid.csv
 preprocessed_reviews.csv	    Xinana.ipynb
'Price pre

Some important guides used:

https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9

https://intellica-ai.medium.com/aspect-based-sentiment-analysis-everything-you-wanted-to-know-1be41572e238

# Load Data

In [ ]:
with open('reviews_detailed.csv') as f:
    reviews_data = pd.read_csv(f)
f.close()

In [ ]:
# To test the code with a sample (comment to run on full dataset)
#reviews_data=reviews_data.sample(frac=0.001, replace=True, random_state=1)

# Data Preparation

[https://www.kdnuggets.com/2018/08/practitioners-guide-processing-understanding-text-2.html](https://www.kdnuggets.com/2018/08/practitioners-guide-processing-understanding-text-2.html)

https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

In [ ]:
# Drop columns 'date', 'reviewer_Id', 'reviewer_name'
reviews_data.drop(['date', 'reviewer_id', 'reviewer_name'], axis = 1, inplace=True)
reviews_data.head()

,listing_id,id,comments
128037,4413032,462771700,We stayed a great weekend at Eric and Karen’s ...
267336,14170174,101467737,Anouk's home is a lovely apartment situated on...
312201,17875133,333275585,Leentje a été très réactive lors de mes questi...
371403,23165393,502901319,Very lovely flat with a nice garden.\nGood loc...
73349,1723551,335873405,Die Unterkunft liegt zentru (Hidden by Airbnb)...


In [ ]:
# Examine missing values:
for col in reviews_data.columns:
    print(col, reviews_data[col].isnull().sum())

listing_id 0
id 0
comments 0


In [ ]:
# Drop rows that have no comments
cleaned_reviews_data = reviews_data[reviews_data['comments'].notnull()]
for col in cleaned_reviews_data.columns:
    print(col, cleaned_reviews_data[col].isnull().sum())

listing_id 0
id 0
comments 0


## Treatment of non-english reviews

In [ ]:

# Identify review language
DetectorFactory.seed = 0

def lang(text):
  language = ""
  try:
    language = detect(text)
  except:
    language = 'undefined'
  return language

cleaned_reviews_data['langs'] = cleaned_reviews_data['comments'].apply(lambda x: lang(x))
cleaned_reviews_data.head()

,listing_id,id,comments,langs
128037,4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en
267336,14170174,101467737,Anouk's home is a lovely apartment situated on...,en
312201,17875133,333275585,Leentje a été très réactive lors de mes questi...,fr
371403,23165393,502901319,Very lovely flat with a nice garden.\nGood loc...,en
73349,1723551,335873405,Die Unterkunft liegt zentru (Hidden by Airbnb)...,de


In [ ]:
cleaned_reviews_data.shape

(441, 4)

In [ ]:
# Drop non-english reviews
cleaned_reviews_data = cleaned_reviews_data[cleaned_reviews_data.langs == 'en']
cleaned_reviews_data.head()

,listing_id,id,comments,langs
128037,4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en
267336,14170174,101467737,Anouk's home is a lovely apartment situated on...,en
371403,23165393,502901319,Very lovely flat with a nice garden.\nGood loc...,en
117583,3902098,141089737,We were 11 girls and had a perfect weekend in ...,en
21440,570753,335756193,We had a great time staying in the beautiful r...,en


In [ ]:
cleaned_reviews_data.shape

(349, 4)

## Normalization of text in reviews

In [ ]:
reviews_data = cleaned_reviews_en["comments"] 

In [ ]:
# ignore reviews where review length == 1 like P,V, . 

single_character=[]
for r in reviews_data:
    if len(str(r)) > 1:
        single_character.append(r)
    else:
        single_character.append("")

In [ ]:
print(len(single_character))

349


In [ ]:
# remove Accented characters from text 

remove_Accented_characters=[]

for ri in single_character:
    ri=str(ri)
    s = ri.replace('â', '')
    text = unicodedata.normalize('NFD', s)\
       .encode('ascii', 'ignore')\
       .decode("utf-8")
    remove_Accented_characters.append(text)
    
# convert short forms to full forms
    
normalize=[]
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

for sf in remove_Accented_characters:
    sf=str(sf)
    sf=sf.lower()
    d = { "fab": "fabulous", "thx": "thanks","gud":"good","gr8":"great","iluvit":"i love it","gret":"great","wonderfull":"wonderful"}
    e=replace_all(sf, d)
    normalize.append(e)

In [ ]:
# ignore rows where only irregular english words come i.e row 13 and 14 

spell=[]
d = enchant.Dict("en_US")
for sp in normalize:
    sp=str(sp)
    sp=sp.lower()
    
    score=[]
    err = re.sub(r'[^\w\s]','',sp)
    word=err.split(" ")
    for w in word:
        if w != '':
            w1=w.lower()
            w2=w.upper()
            v=d.check(w)
            v1=d.check(w1)
            v2=d.check(w2)
            if v == False and v1 == False and v2 == False:
                sc=0
            if v == True or v1 == True or v2 == True:
                sc=1
            score.append(sc)
    suM=sum(score)
    if suM != 0:
        spell.append(sp)
    else:
        spell.append("non-english")

# Extraction of opinion units


In [ ]:
### ignore rows where only irregular english words come i.e row 13 and 14 

sentence=[]
reviews=[]
aspects=[]
 
for review in spell:
    review=str(review)
    doc = nlp(review)
    sub_list=[]
    for sent in doc.sents:
        a=sent.string.strip()
        io=(a,review)
        sub_list.append(io)
        
    sentence.append(sub_list)

 
for ty in sentence:
     
    for tz in ty:
        
        reviews.append(tz[1])
        aspects.append(tz[0])

In [ ]:
review_ = pd.DataFrame({'REVIEWS' : reviews,'OPINION_UNITS' : aspects})
review_.head()

,REVIEWS,OPINION_UNITS
0,we stayed a great weekend at eric and karens h...,we stayed a great weekend at eric and karens h...
1,we stayed a great weekend at eric and karens h...,karen welcomed us even late on our arrival day.
2,we stayed a great weekend at eric and karens h...,"we rented them two very good bikes, the city i..."
3,we stayed a great weekend at eric and karens h...,"the location is really good, 20 min from the t..."
4,we stayed a great weekend at eric and karens h...,"10 min by bike, \nvery quiet area, living area"


In [ ]:
words=[]
part_of_speech=["NOUN","ADV","ADJ","PROPN"]
helping_verb=["is","am","are","was","were","has","have","had","do","does","did","be"]
for rev in aspects:
    rev=str(rev)
    rev=rev.lower()
    doc = nlp(rev)
    # tokenize reviews and remove punctuation, stopwords and lemmatize 
    reviews = [(token.lemma_,token.tag_) for token in doc if not token.is_punct and (token.pos_ in part_of_speech or token.text == "no") and token.text not in helping_verb]
    words.append(reviews)

In [ ]:
review_['spacy_tag'] = words
review_.head()

,REVIEWS,OPINION_UNITS,spacy_tag
0,we stayed a great weekend at eric and karens h...,we stayed a great weekend at eric and karens h...,"[(great, JJ), (weekend, NN), (eric, NNP), (kar..."
1,we stayed a great weekend at eric and karens h...,karen welcomed us even late on our arrival day.,"[(karen, NNP), (even, RB), (late, RB), (arriva..."
2,we stayed a great weekend at eric and karens h...,"we rented them two very good bikes, the city i...","[(very, RB), (good, JJ), (bike, NNS), (city, N..."
3,we stayed a great weekend at eric and karens h...,"the location is really good, 20 min from the t...","[(location, NN), (really, RB), (good, JJ), (mi..."
4,we stayed a great weekend at eric and karens h...,"10 min by bike, \nvery quiet area, living area","[(min, NNP), (bike, NN), (very, RB), (quiet, J..."


# Convert Spacy tags to SentiWordNet accepted tags
* The POS tagger gives you a different tag than the one senti_synset accepts. Use the following to convert to synset notation.

In [ ]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

# Calculate sentiment scores

## Read Positive and Negative Words from two text files

In [ ]:
with  open ('positive-words.txt','r', encoding='latin-1') as positive_file:
    positive_words=[p_word.strip() for p_word in positive_file]
with  open ('negative-words.txt','r', encoding='latin-1')  as negative_file:
    negative_words=[n_word.strip() for n_word in negative_file] 

## For Sentiment Score use following four lexicon

##### SentiWordNet
https://github.com/aesuli/sentiwordnet

##### AFINN
https://pypi.org/project/afinn/

##### Setiment Vader
https://www.nltk.org/api/nltk.sentiment.html

##### Kaggle Text Files
Two Text Files https://www.kaggle.com/harshaiitj08/positive-and-negative-words#positive-words.txt

In [ ]:
score=[]
for u in words:
    sub_list=[]
    for word,tag in u:
        
        wn_tag = penn_to_wn(tag)
        
### Find synsets of word using wordnet

        synsets = wn.synsets(word, pos=wn_tag)
        if synsets == []:
            sentiment = 0.0

        synsets_word=[]
        for sets in synsets:
            word_search=re.search('(?P<word>(\w+)*)',sets.name())
            if word == word_search.group("word"):
                synset = sets
                synsets_word.append(synset)

        if synsets_word == []: 
            synset0_sentiment = 0.0

        if synsets_word != []: 
            ab=synsets_word[0]
            
### Calculate synset sentiment score

            swn_synset = swn.senti_synset(ab.name())
            synset0_sentiment = swn_synset.pos_score() - swn_synset.neg_score()

            if synset0_sentiment != 0.0:
                if word in ["fee","fees","hike","tourist","fairy","tale"]:
                    sub_list.append(0.0)

                    
                if word in ["not","no"]:
                    synset0_sentiment = -1.0
                
                sc=str(synset0_sentiment)

                if sc[0] == "-" and word in positive_words:
                    synset0_sentiment = 0.5
                elif sc[0] != "-" and word in negative_words:
                    synset0_sentiment = -0.5
                    sub_list.append(synset0_sentiment)
                
                else:
                    sub_list.append(synset0_sentiment)                 

### if synsets[0],synsets[1] score then 0 then calculate score by using AFINN 

        if synset0_sentiment == 0.0:
            if word in ["help","top","reward"]:continue
            af_score=af.score(word)
            if af_score != 0.0:
                sub_list.append(af_score)

                
### If synsets, and AFINN score = 0 then use positive and negative word read from files to score words

            if af_score == 0.0:
                if word in positive_words:
                    positive_score = 1.0
                    sub_list.append(positive_score)


                if word in negative_words:
                    negative_score = -1.0
                    sub_list.append(negative_score)

                    
### If word score is zero then use sentiment.vader to calculate score

                if word not in positive_words and word not in negative_words:
                    ss = senti.polarity_scores(word)  
                    if ss["pos"] > 0.0:
                        positive = 1.0
                        sub_list.append(positive)

                    if ss["neg"] > 0.0:
                        negative = -1.0
                        sub_list.append(negative)

                    if ss["neu"] > 0.0:
                        neutral = 0.000
                        sub_list.append(neutral)

                    if ss["pos"] == 0.0 and ss["neg"] == 0.0 and ss["neu"] == 0.0:
                        sub_list.append(0.0)

    sum_=sum(sub_list)
    score.append(sum_)

## Categorize score into Positive, Negative, Neutral

In [ ]:
review_polarity=[]
review_score=[]

for score_ in score:
     

    if score_ not in [0,0,0] and score_ > 0.0 :
         
        polarity = "Positive"
         
    if score_ < 0.0:
        polarity = "Negative"
    if score_ in [0,0.0]:
        polarity = "Neutral"
        

    review_score.append(score_)
    review_polarity.append(polarity)

In [ ]:
review_.insert(1, "Sentiment Score", review_score, True)    
review_.insert(1, "Sentiment Polarity", review_polarity, True)  
review_.head()

,REVIEWS,Sentiment Polarity,Sentiment Score,OPINION_UNITS,spacy_tag
0,we stayed a great weekend at eric and karens h...,Positive,3.50,we stayed a great weekend at eric and karens h...,"[(great, JJ), (weekend, NN), (eric, NNP), (kar..."
1,we stayed a great weekend at eric and karens h...,Neutral,0.00,karen welcomed us even late on our arrival day.,"[(karen, NNP), (even, RB), (late, RB), (arriva..."
2,we stayed a great weekend at eric and karens h...,Positive,3.75,"we rented them two very good bikes, the city i...","[(very, RB), (good, JJ), (bike, NNS), (city, N..."
3,we stayed a great weekend at eric and karens h...,Positive,0.75,"the location is really good, 20 min from the t...","[(location, NN), (really, RB), (good, JJ), (mi..."
4,we stayed a great weekend at eric and karens h...,Neutral,0.00,"10 min by bike, \nvery quiet area, living area","[(min, NNP), (bike, NN), (very, RB), (quiet, J..."


In [ ]:
review_.shape

(1542, 5)

# Join results with listing dataset

In [ ]:
review_with_polarity = review_

In [ ]:
# First I need the reviews = opinion_units
join_id = cleaned_reviews_data["comments"]

In [ ]:
## ignore reviews where review length == 1 like P,V, . 

single_character=[]
for r in join_id:
    if len(str(r)) > 1:
        single_character.append(r)
    else:
        single_character.append("")


### remove Accented characters from text 

remove_Accented_characters=[]

for ri in single_character:
    ri=str(ri)
    s = ri.replace('â', '')
    text = unicodedata.normalize('NFD', s)\
       .encode('ascii', 'ignore')\
       .decode("utf-8")
    remove_Accented_characters.append(text)
    
### convert short forms to full forms
    
normalize=[]
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

for sf in remove_Accented_characters:
    sf=str(sf)
    sf=sf.lower()
    d = { "fab": "fabulous", "thx": "thanks","gud":"good","gr8":"great","iluvit":"i love it","gret":"great","wonderfull":"wonderful"}
    e=replace_all(sf, d)
    normalize.append(e)

spell=[]
d = enchant.Dict("en_US")
for sp in normalize:
    sp=str(sp)
    sp=sp.lower()
    
    score=[]
    err = re.sub(r'[^\w\s]','',sp)
    word=err.split(" ")
    for w in word:
        if w != '':
            w1=w.lower()
            w2=w.upper()
            v=d.check(w)
            v1=d.check(w1)
            v2=d.check(w2)
            if v == False and v1 == False and v2 == False:
                sc=0
            if v == True or v1 == True or v2 == True:
                sc=1
            score.append(sc)
    suM=sum(score)
    if suM != 0:
        spell.append(sp)
    else:
        spell.append("non-english")

In [ ]:
cleaned_reviews_data['join_id'] = spell

In [ ]:
cleaned_reviews_data.head()

,listing_id,id,comments,langs,join_id
128037,4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...
267336,14170174,101467737,Anouk's home is a lovely apartment situated on...,en,anouk's home is a lovely apartment situated on...
371403,23165393,502901319,Very lovely flat with a nice garden.\nGood loc...,en,very lovely flat with a nice garden.\ngood loc...
117583,3902098,141089737,We were 11 girls and had a perfect weekend in ...,en,we were 11 girls and had a perfect weekend in ...
21440,570753,335756193,We had a great time staying in the beautiful r...,en,we had a great time staying in the beautiful r...


In [ ]:
# now merge cleaned_revies_data that has the ids with review_with_polarity which has the scores
new_df = review_with_polarity.merge(cleaned_reviews_data,left_on='REVIEWS',right_on='join_id')

In [ ]:
new_df.head()

,REVIEWS,Sentiment Polarity,Sentiment Score,OPINION_UNITS,spacy_tag,listing_id,id,comments,langs,join_id
0,we stayed a great weekend at eric and karens h...,Positive,3.50,we stayed a great weekend at eric and karens h...,"[(great, JJ), (weekend, NN), (eric, NNP), (kar...",4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...
1,we stayed a great weekend at eric and karens h...,Neutral,0.00,karen welcomed us even late on our arrival day.,"[(karen, NNP), (even, RB), (late, RB), (arriva...",4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...
2,we stayed a great weekend at eric and karens h...,Positive,3.75,"we rented them two very good bikes, the city i...","[(very, RB), (good, JJ), (bike, NNS), (city, N...",4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...
3,we stayed a great weekend at eric and karens h...,Positive,0.75,"the location is really good, 20 min from the t...","[(location, NN), (really, RB), (good, JJ), (mi...",4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...
4,we stayed a great weekend at eric and karens h...,Neutral,0.00,"10 min by bike, \nvery quiet area, living area","[(min, NNP), (bike, NN), (very, RB), (quiet, J...",4413032,462771700,We stayed a great weekend at Eric and Karen’s ...,en,we stayed a great weekend at eric and karens h...


In [ ]:
sentiment_scores = new_df.groupby('listing_id', as_index=False)['Sentiment Score'].mean()

In [ ]:
sentiment_scores = sentiment_scores.rename(columns={'Sentiment Score': 'sentiment_score'})
sentiment_scores.head()

,listing_id,sentiment_score
0,2818,1.541667
1,55256,1.062500
2,68290,0.708333
3,68873,1.421875
4,75382,0.950000


## Merge result with listings dataset

In [ ]:
listings = pd.read_csv('listings.csv')

In [ ]:
new_listings = listings.merge(sentiment_scores,left_on='id',right_on='listing_id')

In [ ]:
new_listings.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,listing_id,sentiment_score
0,2818,https://www.airbnb.com/rooms/2818,20210208155251,2021-02-11,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi<br />...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,https://www.airbnb.com/users/show/3159,Daniel,2008-09-24,"Amsterdam, Noord-Holland, The Netherlands","Upon arriving in Amsterdam, one can imagine as...",NaN,NaN,100%,t,https://a0.muscache.com/im/users/3159/profile_...,https://a0.muscache.com/im/users/3159/profile_...,Indische Buurt,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Amsterdam, North Holland, Netherlands",Oostelijk Havengebied - Indische Buurt,NaN,52.36575,4.94142,Private room in apartment,Private room,2,NaN,1.5 shared baths,1.0,2.0,"[""Hangers"", ""Coffee maker"", ""Paid parking on p...",$59.00,3,1125,3.0,3.0,1125.0,1125.0,3.0,1125.0,NaN,t,29,46,60,139,2021-02-11,278,1,0,2009-03-30,2020-02-14,98.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,t,1,0,1,0,1.92,2818,1.541667
1,55256,https://www.airbnb.com/rooms/55256,20210208155251,2021-02-10,Luminous central room,Room in the old city center near New Market. A...,"It's in the old historical center, just a few ...",https://a0.muscache.com/pictures/03529288-471d...,260785,https://www.airbnb.com/users/show/260785,Lotte,2010-10-13,"Amsterdam, Noord-Holland, The Netherlands","As I'm very keen on my apartment, I like to kn...",NaN,NaN,NaN,f,https://a0.muscache.com/im/users/260785/profil...,https://a0.muscache.com/im/users/260785/profil...,Nieuwmarkt en Lastage,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Amsterdam, North Holland, Netherlands",Centrum-Oost,NaN,52.37126,4.90351,Private room in apartment,Private room,2,NaN,1 shared bath,1.0,1.0,"[""Hot water"", ""Bed linens"", ""Essentials"", ""Kit...",$90.00,1,365,1.0,1.0,1125.0,1125.0,1.0,1125.0,NaN,t,0,0,0,0,2021-02-10,179,0,0,2011-02-20,2019-12-30,92.0,9.0,9.0,9.0,10.0,10.0,8.0,NaN,t,1,0,1,0,1.47,55256,1.062500
2,68290,https://www.airbnb.com/rooms/68290,20210208155251,2021-02-10,Rebel - Private Room,★ Location - The best<br />★ Room - Recently r...,My home is in a 16th century monumental house ...,https://a0.muscache.com/pictures/1eb1ad0f-87e6...,336950,https://www.airbnb.com/users/show/336950,Manuel,2011-01-06,"Amsterdam, Noord-Holland, The Netherlands",I am an easygoing and an open-minded guy that ...,within a day,50%,100%,t,https://a0.muscache.com/im/users/336950/profil...,https://a0.muscache.com/im/users/336950/profil...,De Wallen,2.0,2.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Amsterdam, NH, Netherlands",Centrum-West,NaN,52.36887,4.89523,Private r

In [ ]:
new_listings['polarity'] = new_listings['sentiment_score'].apply(lambda x: 'Positive' if x > 0 else ('Neutral' if x == 0 else 'Negative'))

In [ ]:
new_listings = new_listings.drop('listing_id', 1)

## Save the new dataframe

In [ ]:
#pickle
new_listings.to_pickle('new_listings.pickle')

In [52]:
#------------------------------ END OF CODE -----------------------------------#